In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import random
from typing import List, Dict
from functools import partial
import pickle
import math
from math import comb
import re

pd.set_option("display.max_rows", 83)
pd.set_option("display.max_columns", 83)

In [2]:
from helpers import RANKS, SUITS, RANK_PAIRS, SMALL_STRAIGHT_RANKS, BIG_STRAIGHT_RANKS, TOTAL_CARDS_NUM, EACH_RANK_COUNT, EACH_SUIT_COUNT, RANK_PAIRS_DESCENDING

from Card import Card
from Deck import Deck
from simulate import init_counts

from Solver import Solver, combinations

In [3]:
p = pd.DataFrame()
p = p.astype(float)

hand = [
    Card("9", "♠"),
    Card("T", "♠"),
    Card("9", "♣"),
    Card("J", "♠"),
    # Card("J", "♦"),
    # Card("Q", "♥")
    Card("T", "♥"),
    Card("J", "♥"),
    Card("Q", "♥"),
    Card("A", "♥")
]

r = list(range(2,25))
for n in r:
    
    if n - len(hand) < 0:
        p.loc[n, :] = None
        continue
    
    solver = Solver(
        hand = hand,
        cards_in_play_num = n
    )
    
    for name, method, kwargs in combinations:
        p.loc[n, name] = getattr(solver, method)(**kwargs)

p

,high_card_9,high_card_T,high_card_J,high_card_Q,high_card_K,high_card_A,pair_9,pair_T,pair_J,pair_Q,pair_K,pair_A,two_pair_T_9,two_pair_J_9,two_pair_J_T,two_pair_Q_9,two_pair_Q_T,two_pair_Q_J,two_pair_K_9,two_pair_K_T,two_pair_K_J,two_pair_K_Q,two_pair_A_9,two_pair_A_T,two_pair_A_J,two_pair_A_Q,two_pair_A_K,small_straight,big_straight,three_9,three_T,three_J,three_Q,three_K,three_A,full_9_T,full_9_J,full_9_Q,full_9_K,full_9_A,full_T_9,full_T_J,full_T_Q,full_T_K,full_T_A,full_J_9,full_J_T,full_J_Q,full_J_K,full_J_A,full_Q_9,full_Q_T,full_Q_J,full_Q_K,full_Q_A,full_K_9,full_K_T,full_K_J,full_K_Q,full_K_A,full_A_9,full_A_T,full_A_J,full_A_Q,full_A_K,quad_9,quad_T,quad_J,quad_Q,quad_K,quad_A,flush_♠,flush_♣,flush_♦,flush_♥,small_poker_♠,small_poker_♣,small_poker_♦,small_poker_♥,big_poker_♠,big_poker_♣,big_poker_♦,big_poker_♥
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.0,1.0,1.0,1.0,0.000000,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
9,1.0,1.0,1.0,1.0,0.250000,1.0,1.0,1.0,1.0,0.187500,0.000000,0.187500,1.0,1.0,1.0,0.187500,0.187500,0.187500,0.000000,0.000000,0.000000,0.000000,0.187500,0.187500,0.187500,0.000000,0.000000,0.250000,0.250000,0.125000,0.125000,0.125000,0.000000,0.000000,0.000000,0.125000,0.125000,0.000000,0.000000,0.000000,0.125000,0.125000,0.000000,0.000000,0.000000,0.125000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0